# Human Bone Fracture Detection

We will detect the type of bone breaks in this project. The plans we will follow is
- ~Make a python virtual environment~
- ~Clean and prepare Data~
- [TODO]Create transfer learning model
- Train and get a decent accuracy using just modifications to head
- Unfreezing and running the model
- Test set accuracy and visualization

In [1]:
!pip install numpy
!pip install pandas
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
!pip install matplotlib
!pip install timm
!pip install scikit-learn

Looking in indexes: https://download.pytorch.org/whl/cpu


### Importing dependencies and database

In [1]:
# imorting reqd libs
from pathlib import Path
import pandas as pd
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# importing transformers and stuff for learning
import timm
import torch.nn as nn
import torch.optim as optim
import os
import shutil
from sklearn.model_selection import train_test_split

from torch.utils.data import WeightedRandomSampler


c:\SSY340_DML\bone_fracture_detection\bfd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imoprting dataset


# Creating paths for imports
data_path = Path.cwd()
folder_path = data_path / "bone.v1i.multiclass"

train_path = folder_path / "train"
val_path = folder_path / "valid"
test_path = folder_path / "test"

train_csv_path = train_path / '_classes.csv'
val_csv_path = val_path / '_classes.csv'
test_csv_path = test_path / '_classes.csv'


df_name = 'filename'

In [3]:
# rename training

train_df = pd.read_csv(train_csv_path)
# train_df.columns = train_df.columns.str.strip()

# prefix = "f1_"
# extension = ".jpg"

# file_list = sorted(os.listdir(train_path))
# new_names = []

# for i, old_name in enumerate(file_list, start=1):
#     if old_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
#         new_name = f"{prefix}{i:04d}{extension}"
#         old_path = os.path.join(train_path, old_name)
#         new_path = os.path.join(train_path, new_name)
#         os.rename(old_path, new_path)
#         new_names.append((old_name, new_name))

# rename_dict = dict(new_names)
# train_df[df_name] = train_df[df_name].map(rename_dict).fillna(train_df[df_name])

# train_df.to_csv(train_csv_path, index=False)

train_df.head()

,filename,Avulsion,Comminuted,Compression-Crush,Dislocation,Fracture,Greenstick,Hairline,Impacted,Intra-articular,Longitudinal,Oblique,Pathological,Spiral,fracture
0,f1_2337.jpg,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,f1_1893.jpg,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,f1_2003.jpg,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,f1_1846.jpg,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,f1_0742.jpg,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [4]:
# rename val

val_df = pd.read_csv(val_csv_path)
# val_df.columns = val_df.columns.str.strip()

# prefix = "f2_"
# extension = ".jpg"

# file_list = sorted(os.listdir(val_path))
# new_names = []

# for i, old_name in enumerate(file_list, start=1):
#     if old_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
#         new_name = f"{prefix}{i:04d}{extension}"
#         old_path = os.path.join(val_path, old_name)
#         new_path = os.path.join(val_path, new_name)
#         os.rename(old_path, new_path)
#         new_names.append((old_name, new_name))

# rename_dict = dict(new_names)
# val_df[df_name] = val_df[df_name].map(rename_dict).fillna(val_df[df_name])

# val_df.to_csv(val_csv_path, index=False)

val_df.head()

,filename,Avulsion,Comminuted,Compression-Crush,Dislocation,Fracture,Greenstick,Hairline,Impacted,Intra-articular,Longitudinal,Oblique,Pathological,Spiral,fracture
0,f2_0234.jpg,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,f2_0073.jpg,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,f2_0295.jpg,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,f2_0241.jpg,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,f2_0205.jpg,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [5]:
# rename test

test_df = pd.read_csv(test_csv_path)
# test_df.columns = test_df.columns.str.strip()

# prefix = "f3_"
# extension = ".jpg"

# file_list = sorted(os.listdir(test_path))
# new_names = []

# for i, old_name in enumerate(file_list, start=1):
#     if old_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
#         new_name = f"{prefix}{i:04d}{extension}"
#         old_path = os.path.join(test_path, old_name)
#         new_path = os.path.join(test_path, new_name)
#         os.rename(old_path, new_path)
#         new_names.append((old_name, new_name))

# rename_dict = dict(new_names)
# test_df[df_name] = test_df[df_name].map(rename_dict).fillna(test_df[df_name])

# test_df.to_csv(test_csv_path, index=False)

test_df.head()

,filename,Avulsion,Comminuted,Compression-Crush,Dislocation,Fracture,Greenstick,Hairline,Impacted,Intra-articular,Longitudinal,Oblique,Pathological,Spiral,fracture
0,f3_0123.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,f3_0122.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,f3_0148.jpg,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,f3_0007.jpg,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,f3_0135.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [6]:
combined_path = folder_path / 'all'
# os.makedirs(combined_path, exist_ok=True)

# # copy into one folder
# for file_name in os.listdir(test_path):
#     if file_name.lower().endswith(('.jpg', '.png', '.bmp')):
#         src_path = os.path.join(test_path, file_name)
#         print
#         dst_path = os.path.join(combined_path, file_name)
#         try:
#             shutil.copy2(src_path, dst_path)  # copy2 preserves metadata
#             # count += 1
#         except Exception as e:
#             print(f"Could not copy {file_name}: {e}")


# combine all datasets into one
# combined_df = pd.concat([train_df, val_df, test_df])
# combined_df = combined_df.drop(columns=['Fracture', 'fracture'])

combined_csv_path = combined_path / '_classes.csv'

# combined_df.to_csv(combined_csv_path, index=False)

combined_df = pd.read_csv(combined_csv_path)

combined_df.head()

,filename,Avulsion,Comminuted,Compression-Crush,Dislocation,Greenstick,Hairline,Impacted,Intra-articular,Longitudinal,Oblique,Pathological,Spiral
0,f1_2337.jpg,1,0,0,0,0,0,0,0,0,0,0,0
1,f1_1893.jpg,1,0,0,0,0,0,0,0,0,0,0,0
2,f1_2003.jpg,0,1,0,0,0,0,0,0,0,0,0,0
3,f1_1846.jpg,0,0,0,0,0,0,1,0,0,0,0,0
4,f1_0742.jpg,1,0,0,0,0,0,0,0,0,0,0,0


In [9]:
class_columns = [col for col in combined_df.columns if col != 'filename']

combined_df['class'] = combined_df[class_columns].idxmax(axis=1)

# print(combined_df['class'].value_counts())



In [10]:
# train_ratio, val_ratio, test_ratio = 0.9, 0.05, 0.05

# train_df, temp_df = train_test_split(
#     combined_df,
#     test_size=(1 - train_ratio),
#     stratify=combined_df['class'],
#     random_state=42
# )

# val_df, test_df = train_test_split(
#     temp_df,
#     test_size=(test_ratio / (test_ratio + val_ratio)),
#     stratify=temp_df['class'],
#     random_state=42
# )

# print("Train class distribution:")
# print(train_df['class'].value_counts())

# # print("\nVal class distribution:")
# # print(val_df['class'].value_counts())

# # print("\nTest class distribution:")
# # print(test_df['class'].value_counts())

train_df, temp_df = train_test_split(
    combined_df, test_size=0.20, stratify=combined_df['class'], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.50, stratify=temp_df['class'], random_state=42
)

# print(train_df['class'].value_counts())
# print(val_df['class'].value_counts())
print(test_df['class'].value_counts())

class
Comminuted           64
Dislocation          47
Compression-Crush    44
Avulsion             41
Greenstick           39
Hairline             39
Impacted             33
Spiral               14
Intra-articular      14
Pathological         12
Oblique               9
Longitudinal          9
Name: count, dtype: int64


In [122]:
# ---- Modified Dataset Class ----
class BoneDataset(Dataset):
    def __init__(self, df, folder_path, transform=None):
        """
        Args:
            df (pd.DataFrame): DataFrame containing 'filename' and 'class' columns.
            folder_path (str or Path): Path to image directory.
            transform (torchvision.transforms): Transformations to apply.
        """
        self.df = df.reset_index(drop=True)
        self.folder_path = Path(folder_path)
        self.transform = transform

        # Get class mappings (string → int)
        self.classes = sorted(self.df['class'].unique())
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.folder_path / row['filename']

        # Load image
        image = Image.open(img_path).convert("L")

        # Convert class name → integer label
        # label = self.class_to_idx[row['class']]
        # label = torch.tensor(label, dtype=torch.long)

        # label = self.df.iloc[idx, 1]      # class label
        label_name = self.df.iloc[idx]['class']
        label_idx = self.class_to_idx[label_name]
        label = torch.tensor(label_idx, dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label


# ---- Define Transforms ----
transform = T.Compose([
    T.Resize((224, 224)),
    T.Grayscale(num_output_channels=3),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5]),
])

# ---- Datasets ----
train_dataset = BoneDataset(train_df, folder_path=combined_path, transform=transform)
valid_dataset = BoneDataset(val_df, folder_path=combined_path, transform=transform)
test_dataset  = BoneDataset(test_df, folder_path=combined_path, transform=transform)


# ---- Dataloaders ----
batch_size = 32
loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
loader_test  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ---- Sanity check ----
for images, labels in loader_valid:
    print(f"Batch images shape: {images.shape}")
    print(f"Batch labels: {labels}")
    break


Batch images shape: torch.Size([32, 3, 224, 224])
Batch labels: tensor([ 1,  2,  0,  2,  1,  4,  3, 10,  1,  2,  1,  8,  1,  1,  6,  1,  8,  5,
        10,  5,  6,  1,  6,  4,  3,  1,  0,  5,  4,  2,  0,  0])


We can use ViT, DeiT, Swin Transformer, or MedViT

MedViT seems relevant but will not be easy to fine tune, and 

AI says we should use DeIT as it is easy to fine tune and will work well with our dataset

ViT would also be valid, but it is more generic

Swin would be a good idea as well but we think we follow AI and start with DeIT

In [260]:
def output_to_label(z):
    c = torch.argmax(z, dim=1)
    return c

In [261]:
def training_loop(model, optimizer, loss_fn, train_loader, val_loader, num_epochs, print_every):
    print("Starting training")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_losses, train_accs, val_losses, val_accs = [], [], [], []

    for epoch in range(1, num_epochs + 1):
        
        # if epoch == 4:
        #     print("Unfreezing last 4 DeiT blocks for fine-tuning...")
        #     for name, param in model.base.blocks[-4:].named_parameters():
        #         param.requires_grad = True
        #     for name, param in model.base.norm.named_parameters():
        #         param.requires_grad = True
        #     optimizer = optim.AdamW(
        #         filter(lambda p: p.requires_grad, model.parameters()),
        #         lr=1e-5,
        #         weight_decay=0.05
        #     )

        model, train_loss, train_acc = train_epoch(
            model, optimizer, loss_fn, train_loader, val_loader, device, print_every
        )
        val_loss, val_acc = validate(model, loss_fn, val_loader, device)
        print(
            f"Epoch {epoch}/{num_epochs}: "
            f"Train loss: {sum(train_loss)/len(train_loss):.3f}, "
            f"Train acc.: {sum(train_acc)/len(train_acc):.3f}, "
            f"Val. loss: {val_loss:.3f}, "
            f"Val. acc.: {val_acc:.3f}"
        )
        train_losses.extend(train_loss)
        train_accs.extend(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
    return model, train_losses, train_accs, val_losses, val_accs


def train_epoch(model, optimizer, loss_fn, train_loader, val_loader, device, print_every):
    # Train:
    model.train()
    train_loss_batches, train_acc_batches = [], []
    num_batches = len(train_loader)
    for batch_index, (x, y) in enumerate(train_loader, 1):
        inputs, labels = x.to(device), y.to(device)
        optimizer.zero_grad()
        z = model.forward(inputs)
        # print("Outputs:", z.shape)
        # print("Targets:", labels.shape)
        labels.long()
        loss = loss_fn(z, labels)
        loss.backward()
        optimizer.step()
        train_loss_batches.append(loss.item())

        hard_preds = output_to_label(z)
        acc_batch_avg = (hard_preds == labels).float().mean().item()
        train_acc_batches.append(acc_batch_avg)

        # If you want to print your progress more often than every epoch you can
        # set `print_every` to the number of batches you want between every status update.
        # Note that the print out will trigger a full validation on the full val. set => slows down training
        if print_every is not None and batch_index % print_every == 0:
            val_loss, val_acc = validate(model, loss_fn, val_loader, device)
            model.train()
            print(
                f"\tBatch {batch_index}/{num_batches}: "
                f"\tTrain loss: {sum(train_loss_batches[-print_every:])/print_every:.3f}, "
                f"\tTrain acc.: {sum(train_acc_batches[-print_every:])/print_every:.3f}, "
                f"\tVal. loss: {val_loss:.3f}, "
                f"\tVal. acc.: {val_acc:.3f}"
            )

    return model, train_loss_batches, train_acc_batches


def validate(model, loss_fn, val_loader, device):
    val_loss_cum = 0
    val_acc_cum = 0
    model.eval()
    with torch.no_grad():
        for batch_index, (x, y) in enumerate(val_loader, 1):
            inputs, labels = x.to(device), y.to(device)
            z = model.forward(inputs)

            labels.long()
            batch_loss = loss_fn(z, labels)
            val_loss_cum += batch_loss.item()
            hard_preds = output_to_label(z)
            acc_batch_avg = (hard_preds == labels).float().mean().item()
            val_acc_cum += acc_batch_avg
    return val_loss_cum / len(val_loader), val_acc_cum / len(val_loader)

In [262]:
# loading DeIT from timm
num_classes = 12
# base_model = timm.create_model(
#     'swin_tiny_patch4_window7_224',
#     pretrained=True,
#     num_classes=num_classes,   # <— forces classification head
#     global_pool='avg'          # <— ensures 7×7 gets averaged
# )

# # base_model.head

# in_features = base_model.head.in_features
# base_model.head = nn.Linear(in_features, num_classes)

# x = torch.randn(32, 3, 224, 224)
# out = base_model(x)
# print(out.shape)

base_model = timm.create_model('deit_small_patch16_224', pretrained=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\SSY340_DML\bone_fracture_detection\bfd\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\toshi\.cache\huggingface\hub\models--timm--deit_small_patch16_224.fb_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer m

In [263]:
# freezing layers

for param in base_model.parameters():
    param.requires_grad = False

In [264]:
# creating base model and head model

base_model.head = nn.Identity() 

class HeadModel(nn.Module):
    def __init__(self, in_features, num_classes):
        super(HeadModel, self).__init__()
        self.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.head(x)
    
head_model = HeadModel(in_features=384, num_classes=12)  

In [265]:
# creating full model

class FullModel(nn.Module):
    def __init__(self, base_model, head_model):
        super(FullModel, self).__init__()
        self.base = base_model
        self.head = head_model


    def forward(self, x):
        x = self.base(x)
        # print(x.shape)
        x = self.head(x)

        return x
    
# base_model.head = head_model

model = FullModel(base_model, head_model)

print(model)

FullModel(
  (base): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (norm): Identity()
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act):

In [266]:
# computing classwise loss weights
class_counts = train_df['class'].value_counts().sort_index()
total = sum(class_counts)
class_weights = [total / c for c in class_counts]  # inverse frequency
# class_weights = 1.0 / torch.tensor(class_counts.values, dtype=torch.float)
class_weights = torch.tensor(class_weights, dtype=torch.float)

loss = nn.CrossEntropyLoss(weight=class_weights)

In [267]:
# training the model

# loss = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5,weight_decay=0.05)
num_epochs = 15
print_every = None

model, train_losses, train_accs, val_losses, val_accs = training_loop(model, optimizer, loss, loader_train, loader_valid, num_epochs, print_every)

Starting training
Epoch 1/15: Train loss: 2.515, Train acc.: 0.119, Val. loss: 2.385, Val. acc.: 0.214
Epoch 2/15: Train loss: 2.270, Train acc.: 0.247, Val. loss: 2.245, Val. acc.: 0.304
Epoch 3/15: Train loss: 2.130, Train acc.: 0.320, Val. loss: 2.145, Val. acc.: 0.339
Epoch 4/15: Train loss: 2.024, Train acc.: 0.359, Val. loss: 2.078, Val. acc.: 0.378
Epoch 5/15: Train loss: 1.916, Train acc.: 0.404, Val. loss: 2.017, Val. acc.: 0.385
Epoch 6/15: Train loss: 1.845, Train acc.: 0.411, Val. loss: 1.976, Val. acc.: 0.413
Epoch 7/15: Train loss: 1.781, Train acc.: 0.434, Val. loss: 1.933, Val. acc.: 0.419
Epoch 8/15: Train loss: 1.724, Train acc.: 0.465, Val. loss: 1.894, Val. acc.: 0.434
Epoch 9/15: Train loss: 1.663, Train acc.: 0.472, Val. loss: 1.875, Val. acc.: 0.440
Epoch 10/15: Train loss: 1.611, Train acc.: 0.495, Val. loss: 1.848, Val. acc.: 0.447
Epoch 11/15: Train loss: 1.576, Train acc.: 0.514, Val. loss: 1.837, Val. acc.: 0.442
Epoch 12/15: Train loss: 1.528, Train acc.: 0

In [ ]:
# plotting results

